# Justin Meyer        CS 5310 Ch. 3 Lab - kNN

In this lab I will do some data preprocessing to control for multicollinearity and then normalize the data set before running the kNN algorithm

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import the Features data sets

df1 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\BG_features.csv")
df2 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\FZ_features.csv")
df3 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\MJ_features.csv")
df4 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\MR_features.csv")
df5 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\YH_features.csv")
Features_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [3]:
# Import the Objects data sets

df1 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\BG_objects.csv")
df2 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\FZ_objects.csv")
df3 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\MJ_objects.csv")
df4 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\MR_objects.csv")
df5 = pd.read_csv("E:\MSDA\MSDA Courses\CS 5310\BPass-ICA-EMD-95features-ws1000\YH_objects.csv")
Objects_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [4]:
# Create the correlation matrix and convert values to absolute values

cor_output = Features_df.corr(method='pearson', min_periods=1).abs()

In [5]:
# Create come logic to quickly remove all values correlated that is above .80

mask = np.triu(np.ones_like(cor_output, dtype=bool))
tri_df = cor_output.mask(mask)

to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.80)]

reduced_df = Features_df.drop(to_drop, axis=1)

In [6]:
# Normalize the features using the preprocessing MinMaxScaler from sklearn

from sklearn import preprocessing
x = reduced_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_df = pd.DataFrame(x_scaled)

In [7]:
# Add back in the column headers

col_names = pd.DataFrame(reduced_df.columns)
normalized_df.columns = col_names

In [8]:
# Fixes an issue with the indexing for the Objects dataframe

Objects_df.reset_index(drop=True, inplace=True)

In [9]:
# Combine the dataframes

Full_df = Objects_df.join(normalized_df)
print(Full_df)

      # Object  (beta0,)  (gamma0,)  (hjorth_complexity0,)  (spec_entropy0,)  \
0            3  0.157238   0.229738               0.035377          0.217393   
1            2  0.151668   0.223412               0.030406          0.247178   
2            5  0.121261   0.330426               0.014179          0.081143   
3            1  0.151377   0.216478               0.041233          0.203189   
4            1  0.177493   0.222989               0.086776          0.300749   
...        ...       ...        ...                    ...               ...   
1265         2  0.075409   0.199479               0.024074          0.176828   
1266         1  0.087652   0.235093               0.012779          0.077244   
1267         2  0.098463   0.202751               0.050067          0.196071   
1268         1  0.704196   0.623125               0.043094          0.303839   
1269         3  0.112723   0.245239               0.021689          0.144053   

      (approx_entropy0,)  (Hurst0,)  (a

In [10]:
# Split the data into features and labels

X = np.array(Full_df.drop(['# Object'],1))
y = np.array(Full_df['# Object'])

In [11]:
# Split the data into training data set and testing data set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [12]:
# Train the kNN model and output the crosstab

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)
nuro_test_pred = neigh.predict(X_test)
pd.crosstab(y_test, nuro_test_pred)

col_0,1,2,3,4,5
row_0,,,,,
1,19,8,15,6,7
2,22,11,7,8,8
3,18,8,7,11,7
4,19,7,10,6,4
5,15,12,5,5,9


In [13]:
# How accurate is the model?

accuracy = neigh.score(X_test, y_test)
print(accuracy)

0.2047244094488189


# Conclusion

The model was only 20% accurate in predicting which object belonged to which set of features